# TsRelBreakAdjust

Module that combines the break detection/testing and correction with the implemented methods. It also contains input data evaluations before adjustment and verifications of results after adjustment.

In [1]:
from set_up import *
pd.options.display.max_rows=7
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# Specific for this module
from pybreaks.break_adjust import TsRelBreakAdjust

In [2]:
# this cell loads either test data with properties as set by the user or real data and allows to introduce a break
def load_data(real=False, trend=0.01, bias=0.1, breaktime=datetime(2012,7,1), 
              size=(0.2, 1.1), resample=False):
    if real:
        CAN, REF = load_real_data(707393, breaktime, size, resample)
    else:
        CAN, REF = load_test_data(trend, bias, breaktime, size, resample, seed=1234)
    return CAN, REF

In [3]:
bias_corr_methods = ['linreg', 'mean_std', 'min_max', 'cdf_match']
@interact(BIAS_CORR_METHOD=bias_corr_methods, ADJUST_GROUP=[0,1], 
          BREAK_ADD=(-0.4, 0.4, 0.05),BREAK_MULT=(0.7, 1.3, 0.05), 
          RESAMPLE=['NONE', 'INPUT', 'OUTPUT'])

def BreakAdjustLMP(USE_REAL_DATA=False, BIAS_CORR_METHOD='cdf_match', 
                   ADJUST_TF_ONLY=False, ADJUST_GROUP=0, BREAK_ADD=0.2, BREAK_MULT=1.1, RESAMPLE='NONE'):
    
    BREAKTIME=datetime(2012,7,1)
    TIMEFRAME = [datetime(2011,1,1), datetime(2017,1,1)]
    size=(BREAK_ADD, BREAK_MULT)
    
    resample = True if RESAMPLE=='INPUT' else False
    CAN, REF = load_data(USE_REAL_DATA, 0.01, 0.1, BREAKTIME, size, resample)
    
    
    test_kwargs = dict([('test_resample', ('M', 0.3)),
                        ('mean_test', 'wilkoxon'),
                        ('var_test', 'scipy_fligner_killeen'),
                        ('alpha', 0.01),
                        ('test_check_min_data', 5),
                        ('test_check_spearR_sig', [0, 0.01])])
    
    adjmodel_kwargs = dict([('regress_resample', None),
                            ('model_intercept', True),
                            ('filter', ('both', 5))])

    adjfct_kwargs = dict([('corrections_from_core', True),
                          ('values_to_adjust_freq', 'D'),
                          ('resample_corrections', True),
                          ('interpolation_method', 'linear')])
        
                        
    adjust = TsRelBreakAdjust(CAN, REF, BREAKTIME, adjustment_method='LMP', candidate_flags=None, 
                              timeframe=TIMEFRAME, bias_corr_method=BIAS_CORR_METHOD, adjust_tf_only=ADJUST_TF_ONLY, 
                              adjust_group=ADJUST_GROUP, input_resolution='M' if resample else 'D',
                              adjust_check_pearsR_sig=(0,1), adjust_check_min_group_range=0, 
                              adjust_check_ppcheck=(False,False), create_model_plots=False, test_kwargs=test_kwargs,
                              adjmodel_kwargs=adjmodel_kwargs)

    print(adjust.isbreak, adjust.breaktype)
    
    adjust.test_and_adjust(min_iter=None, max_iter=3, correct_below_0=True, **adjfct_kwargs)
    
    data_adjusted, stillbreak, error = adjust.get_adjusted()
    
    print(stillbreak, error)
    
    test_results_b4, models_ifirst, test_results_aft, models_ilast, \
    group_stats, group_metrics, metrics_change, checkstats = adjust.get_results()
    
    adjust.plot_adj_ts()
    ##adjust.plot_adj_stats_ts()
    
    ##plotting##
    fig, (ax1) = plt.subplots(1,1, figsize=(15,4)) 
    if RESAMPLE == 'OUTPUT':
        adjust.df_original.resample('M').mean().plot(ax=ax1)
    else:
        adjust.df_original.plot(ax=ax1)
   

aW50ZXJhY3RpdmUoY2hpbGRyZW49KENoZWNrYm94KHZhbHVlPUZhbHNlLCBkZXNjcmlwdGlvbj11J1VTRV9SRUFMX0RBVEEnKSwgVGV4dCh2YWx1ZT11J0xNUCcsIGRlc2NyaXB0aW9uPXUnQUTigKY=


In [4]:
bias_corr_methods = ['linreg', 'mean_std', 'min_max', 'cdf_match']
@interact(BIAS_CORR_METHOD=bias_corr_methods, ADJUST_GROUP=[0,1], 
          BREAK_ADD=(-0.4, 0.4, 0.05),BREAK_MULT=(0.7, 1.3, 0.05), 
          RESAMPLE=['NONE', 'INPUT', 'OUTPUT'])

def BreakAdjustHOM(USE_REAL_DATA=False, BIAS_CORR_METHOD='cdf_match', 
                   ADJUST_TF_ONLY=False, ADJUST_GROUP=0, BREAK_ADD=0.2, 
                   BREAK_MULT=1.1, RESAMPLE='NONE'):
    
    BREAKTIME=datetime(2012,7,1)
    TIMEFRAME = [datetime(2011,1,1), datetime(2017,1,1)]
    size=(BREAK_ADD, BREAK_MULT)
    
    resample = True if RESAMPLE=='INPUT' else False
    CAN, REF = load_data(USE_REAL_DATA, 0.01, 0.1, BREAKTIME, size, resample)
    
    
    test_kwargs = dict([('test_resample', ('M', 0.3)),
                        ('mean_test', 'wilkoxon'),
                        ('var_test', 'scipy_fligner_killeen'),
                        ('alpha', 0.01),
                        ('test_check_min_data', 5),
                        ('test_check_spearR_sig', [0, 0.01])])

    adjmodel_kwargs = dict([('regress_resample', None),
                            ('filter', ('both', 5)),
                            ('poly_orders', [1,2]),
                            ('select_by', 'R'),
                            ('cdf_types', None)])

    adjfct_kwargs = {'alpha': 0.6,
                     'use_separate_cdf': False,
                     'from_bins': False}
    
    adjust = TsRelBreakAdjust(CAN, REF, BREAKTIME, adjustment_method='HOM', candidate_flags=None, 
                              timeframe=TIMEFRAME, bias_corr_method=BIAS_CORR_METHOD, adjust_tf_only=ADJUST_TF_ONLY, 
                              adjust_group=ADJUST_GROUP, input_resolution='M' if resample else 'D',
                              adjust_check_pearsR_sig=(0,1), adjust_check_min_group_range=0, 
                              adjust_check_ppcheck=(False,False), create_model_plots=True, test_kwargs=test_kwargs,
                              adjmodel_kwargs=adjmodel_kwargs)

    print(adjust.isbreak, adjust.breaktype)
    
    adjust.test_and_adjust(min_iter=None, max_iter=3, correct_below_0=True, **adjfct_kwargs)
    
    data_adjusted, stillbreak, error = adjust.get_adjusted()
    
    print(stillbreak, error)
    
    test_results_b4, models_ifirst, test_results_aft, models_ilast, \
    group_stats, group_metrics, metrics_change, checkstats = adjust.get_results()
                              
    
    ##plotting##
    fig, (ax1) = plt.subplots(1,1, figsize=(15,4)) 
    if RESAMPLE == 'OUTPUT':
        adjust.df_original.resample('M').mean().plot(ax=ax1)
    else:
        adjust.df_original.plot(ax=ax1)
        
    '''
    
    fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(15,4))
    lmp.plot_models(axs=[ax1, ax2])
    
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,4))
    lmp.plot_adjustments(ax=ax2)
    '''
   

aW50ZXJhY3RpdmUoY2hpbGRyZW49KENoZWNrYm94KHZhbHVlPUZhbHNlLCBkZXNjcmlwdGlvbj11J1VTRV9SRUFMX0RBVEEnKSwgRHJvcGRvd24oZGVzY3JpcHRpb249dSdCSUFTX0NPUlJfTUXigKY=


In [5]:
bias_corr_methods = ['linreg', 'mean_std', 'min_max', 'cdf_match']
@interact(BIAS_CORR_METHOD=bias_corr_methods, ADJUST_GROUP=[0,1], 
          BREAK_ADD=(-0.4, 0.4, 0.05),BREAK_MULT=(0.7, 1.3, 0.05), 
          RESAMPLE=['NONE', 'INPUT', 'OUTPUT'])

def BreakAdjustQCM(USE_REAL_DATA=False, BIAS_CORR_METHOD='cdf_match', 
                   ADJUST_TF_ONLY=False, ADJUST_GROUP=0, BREAK_ADD=0.2, 
                   BREAK_MULT=1.1, RESAMPLE='NONE'):
    
    BREAKTIME=datetime(2012,7,1)
    TIMEFRAME = [datetime(2010,1,1), datetime(2017,1,1)]
    size=(BREAK_ADD, BREAK_MULT)
    
    resample = True if RESAMPLE=='INPUT' else False
    CAN, REF = load_data(USE_REAL_DATA, 0.01, 0.1, BREAKTIME, size, resample)
    
    
    test_kwargs = dict([('test_resample', ('M', 0.3)),
                        ('mean_test', 'wilkoxon'),
                        ('var_test', 'scipy_fligner_killeen'),
                        ('alpha', 0.01),
                        ('test_check_min_data', 5),
                        ('test_check_spearR_sig', [0, 0.01])])
    
    adjmodel_kwargs = dict([('categories', 4),
                            ('first_last', 'formula'),
                            ('fit', 'mean')])

    adjfct_kwargs = {'interpolation_method': 'cubic'}
    

                        
    adjust = TsRelBreakAdjust(CAN, REF, BREAKTIME, adjustment_method='QCM', candidate_flags=None, 
                              timeframe=TIMEFRAME, bias_corr_method=BIAS_CORR_METHOD, adjust_tf_only=ADJUST_TF_ONLY, 
                              adjust_group=ADJUST_GROUP, input_resolution='M' if resample else 'D',
                              adjust_check_pearsR_sig=(0,1), adjust_check_min_group_range=0, 
                              adjust_check_ppcheck=(False,False), create_model_plots=True, test_kwargs=test_kwargs,
                              adjmodel_kwargs=adjmodel_kwargs)

    print(adjust.isbreak, adjust.breaktype)
    
    adjust.test_and_adjust(min_iter=None, max_iter=3, correct_below_0=True, **adjfct_kwargs)
    
    data_adjusted, stillbreak, error = adjust.get_adjusted()
    
    print(stillbreak, error)
    
    test_results_b4, models_ifirst, test_results_aft, models_ilast, \
    group_stats, group_metrics, metrics_change, checkstats = adjust.get_results()
                              
    
    ##plotting##
    fig, (ax1) = plt.subplots(1,1, figsize=(15,4)) 
    if RESAMPLE == 'OUTPUT':
        adjust.df_original.resample('M').mean().plot(ax=ax1)
    else:
        adjust.df_original.plot(ax=ax1)
        
    '''
    
    fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(15,4))
    lmp.plot_models(axs=[ax1, ax2])
    
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,4))
    lmp.plot_adjustments(ax=ax2)
    '''
   

aW50ZXJhY3RpdmUoY2hpbGRyZW49KENoZWNrYm94KHZhbHVlPUZhbHNlLCBkZXNjcmlwdGlvbj11J1VTRV9SRUFMX0RBVEEnKSwgRHJvcGRvd24oZGVzY3JpcHRpb249dSdCSUFTX0NPUlJfTUXigKY=
